# PyMISP - An interactive tutorial: Basics

## Connecting to MISP
### Your configuration

In [3]:
# The URL of the MISP instance to connect to
misp_url = 'http://127.0.0.1:9090/'
# Can be found in the MISP web interface under 
# http://+MISP_URL+/users/view/me -> Authkey
misp_key = 'btm3o1j6SzKUEsHiNz0vTMYzPfcc5eIKpfaWFADj'
# Should PyMISP verify the MISP certificate
misp_verifycert = False

In [4]:
from pymisp import PyMISP

misp = PyMISP(misp_url, misp_key, misp_verifycert)

## Creating a MISP Event

### Directly

In [3]:
event = misp.new_event(distribution=1,
               threat_level_id=1,
               analysis=1,
               info="Event from notebook")
print("Event id: %s" % event['Event']['id'])

Event id: 16


### Using the MISPEvent constructor

In [4]:
from pymisp import MISPEvent

event_obj = MISPEvent()
event_obj.distribution = 1
event_obj.threat_level_id = 1
event_obj.analysis = 1
event_obj.info = "Event from notebook 2"
event = misp.add_event(event_obj)
event_id = event['Event']['id']
print("Event id: %s" % event_id)

Event id: 17


## Fetching a MISP Event

In [9]:
# Fetch by ID
event = misp.get_event(event_id)
print(event)

{'Event': {'attribute_count': '1', 'published': False, 'orgc_id': '1', 'disable_correlation': False, 'id': '17', 'Orgc': {'uuid': '5aa4dd31-42e0-4ebb-a65f-776c88efcce7', 'id': '1', 'name': 'ORGNAME'}, 'ShadowAttribute': [], 'threat_level_id': '1', 'info': 'Event from notebook 2', 'analysis': '1', 'date': '2018-03-28', 'org_id': '1', 'publish_timestamp': '0', 'uuid': '5abb3fc8-8b04-4f1f-9454-0eba0a00020f', 'Org': {'uuid': '5aa4dd31-42e0-4ebb-a65f-776c88efcce7', 'id': '1', 'name': 'ORGNAME'}, 'sharing_group_id': '0', 'distribution': '1', 'Attribute': [{'ShadowAttribute': [], 'uuid': '5abb4064-23e8-4087-83ed-0cba0a00020f', 'category': 'Network activity', 'event_id': '17', 'deleted': False, 'disable_correlation': False, 'id': '4928', 'sharing_group_id': '0', 'object_id': '0', 'distribution': '5', 'type': 'ip-src', 'to_ids': False, 'timestamp': '1522221156', 'value': '8.8.8.8', 'comment': '', 'object_relation': None}], 'proposal_email_lock': False, 'locked': False, 'RelatedEvent': [{'Event'

## Adding Attribute to an event

### Adding directly

In [10]:
attr_type = "ip-src"
value = "8.8.8.8"
category = "Network activity"
to_ids = False
proposal = False
updated_event = misp.add_named_attribute(event,
                                 attr_type,
                                 value,
                                 category=category,
                                 to_ids=to_ids,
                                 proposal=proposal)
print(updated_event)

[{'errors': {'value': ['A similar attribute already exists for this event.']}, 'message': 'Could not add Attribute', 'url': '/attributes/add', 'name': 'Could not add Attribute'}]


### Using the MISPAttribute constructor

In [11]:
from pymisp import MISPAttribute

# Attribute data already defined
attribute = MISPAttribute()
attribute.type = attr_type
attribute.value = value
attribute.category = category
attribute.proposal = proposal
print(attribute)

<MISPAttribute(type=ip-src, value=8.8.8.8)


In [12]:
# An attribute can also be loaded directly from a JSON
json = '''{
    "type": "ip-dst",
    "value": "127.0.0.1",
    "category": "Network activity",
    "to_ids": false,
    "proposal": false
    }'''

attribute = MISPAttribute()
attribute.from_json(json)
print(attribute)

<MISPAttribute(type=ip-dst, value=127.0.0.1)


### And then, update the event

In [16]:
# Add the attribute to the event
## Fetch the event from MISP
event_dict = misp.get(event_id)['Event']

## Convert it to a PyMISP Event
event = MISPEvent()
event.from_dict(**event_dict)
## Let MISP update the timestamp. Otherwise, MISP will compare the two timestamps 
## (which are equals) and reject the the update
del event.timestamp

## Add the attribute to the event
event.attributes.append(attribute)

## Push the updated event to MISP
event_dict = misp.update(event)
print(event_dict)

{'Event': {'attribute_count': '2', 'published': False, 'orgc_id': '1', 'disable_correlation': False, 'id': '17', 'Orgc': {'uuid': '5aa4dd31-42e0-4ebb-a65f-776c88efcce7', 'id': '1', 'name': 'ORGNAME'}, 'ShadowAttribute': [], 'threat_level_id': '1', 'info': 'Event from notebook 2', 'analysis': '1', 'date': '2018-03-28', 'org_id': '1', 'publish_timestamp': '0', 'uuid': '5abb3fc8-8b04-4f1f-9454-0eba0a00020f', 'Org': {'uuid': '5aa4dd31-42e0-4ebb-a65f-776c88efcce7', 'id': '1', 'name': 'ORGNAME'}, 'sharing_group_id': '0', 'distribution': '1', 'Attribute': [{'ShadowAttribute': [], 'uuid': '5abb4064-23e8-4087-83ed-0cba0a00020f', 'category': 'Network activity', 'event_id': '17', 'deleted': False, 'disable_correlation': False, 'id': '4928', 'sharing_group_id': '0', 'object_id': '0', 'distribution': '5', 'type': 'ip-src', 'to_ids': False, 'timestamp': '1522221156', 'value': '8.8.8.8', 'comment': '', 'object_relation': None}, {'ShadowAttribute': [], 'uuid': '5abb4266-5ba0-4ab9-a2ab-0f360a00020f', '

## Performing search

### Events by their info fields

In [17]:
results = misp.search_index(eventinfo='notebook')
# The data is stored in the field 'response'
results = results['response']

for event in results:
    print(event['id'], ':', event['info'])

17 : Event from notebook 2
16 : Event from notebook
9 : Event from notebook 2
8 : Event from notebook 2
7 : Event from notebook


### Attributes by their values

### Search in all attributes

In [22]:
# Search attributes (specified in controller) where the attribute type is 'ip-src'
# And the to_ids flag is set
response = misp.search(controller='attributes', type_attribute='ip-src', to_ids=False)
# The data is stored in the field 'response'
results = response['response']

# Get all related event
attributes = results['Attribute']
event_ids = set()
for attr in attributes:
    event_ids.add(event_id)

# Fetch all related events
for event_id in event_ids:
    event = misp.get_event(event_id)
    print(event['Event']['info'])

Event from notebook 2


## Creating and adding a MISP Object

In [24]:
from pymisp import MISPObject

object_name = 'email'
object_data = {
    'from': 'admin@admin.test',
    'to': 'admin@foo.bar',
    'subject': 'An email',
}

# Retreive the template ID from the object's name
## Fetch all templates
templates = misp.get_object_templates_list()
## Get the template matching with the object's name
template_id = None
for template in templates:
    cur_name = template['ObjectTemplate']['name']
    cur_id = template['ObjectTemplate']['id']
    if cur_name == object_name:
        template_id = cur_id
        break    
if template_id is None:
    raise Exception('No matching template')

# Create the MISP Object
misp_obj = MISPObject(object_name)
for obj_relation, value in object_data.items():
    misp_obj.add_attribute(obj_relation, **{'value': value})

# Add the object to MISP
response = misp.add_object(event_id,
                           template_id,
                           misp_obj)
print(response)

{'Attribute': [{'uuid': '5abb4316-22a0-446a-a89e-0f260a00020f', 'category': 'Payload delivery', 'event_id': '17', 'distribution': '5', 'value1': 'admin@admin.test', 'disable_correlation': False, 'id': '4933', 'sharing_group_id': '0', 'comment': '', 'deleted': False, 'type': 'email-src', 'to_ids': True, 'timestamp': '1522221846', 'value2': '', 'value': 'admin@admin.test', 'object_id': '1094', 'object_relation': 'from'}, {'uuid': '5abb4316-6594-4ee5-a355-0f260a00020f', 'category': 'Payload delivery', 'event_id': '17', 'distribution': '5', 'value1': 'An email', 'disable_correlation': False, 'id': '4934', 'sharing_group_id': '0', 'comment': '', 'deleted': False, 'type': 'email-subject', 'to_ids': False, 'timestamp': '1522221846', 'value2': '', 'value': 'An email', 'object_id': '1094', 'object_relation': 'subject'}, {'uuid': '5abb4316-80b8-4689-a4e8-0f260a00020f', 'category': 'Network activity', 'event_id': '17', 'distribution': '5', 'value1': 'admin@foo.bar', 'disable_correlation': False, 